In [34]:
import requests

In [35]:
url = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras"

In [36]:
model_path = "model_2024_hairstyle.keras"

In [37]:
response = requests.get(url)
with open(model_path, "wb") as f:
    f.write(response.content)

In [38]:
print("Model downloaded successfully!")

Model downloaded successfully!


In [39]:
import tensorflow as tf

In [40]:
keras_model = tf.keras.models.load_model("model_2024_hairstyle.keras")

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

In [42]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\a_farnoo\AppData\Local\Temp\tmpn23imw5t\assets


INFO:tensorflow:Assets written to: C:\Users\a_farnoo\AppData\Local\Temp\tmpn23imw5t\assets


Saved artifact at 'C:\Users\a_farnoo\AppData\Local\Temp\tmpn23imw5t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2185894123024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2185894123408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2185894123600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2185894124560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2185894124752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2185894125328: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [43]:
tflite_model_path = "model_2024_hairstyle.tflite"

In [44]:
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

In [45]:
print("Model successfully converted to TF-Lite!")

Model successfully converted to TF-Lite!


# Q.1

In [46]:
import os

In [47]:
model_size = os.path.getsize(tflite_model_path) / (1024 * 1024)
print(f"Size of the TF-Lite model: {model_size:.2f} MB")

Size of the TF-Lite model: 76.58 MB


# Q.2

In [48]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

In [49]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [50]:
print("Input details:", input_details)
print("Output details:", output_details)
print("Output index:", output_details[0]['index'])

Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([  1, 200, 200,   3], dtype=int32), 'shape_signature': array([ -1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 13, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output index: 13


# Q.3

In [51]:
pip install pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [52]:
from io import BytesIO
from urllib import request
from PIL import Image

In [53]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

In [54]:
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [55]:
# Download and resize the image
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
target_size = (150, 150)  # Target size from the previous homework
img = download_image(url)
img_resized = prepare_image(img, target_size)
img_resized.show()  # This will display the image

In [56]:
import numpy as np

In [57]:
img_array = np.array(img_resized, dtype='float32')

In [58]:
img_array /= 255.0

In [59]:
img_array = np.expand_dims(img_array, axis=0)

In [60]:
first_pixel_r = img_array[0, 0, 0, 0]  # R channel of the first pixel
print(f"Value in the first pixel, R channel: {first_pixel_r:.2f}")

Value in the first pixel, R channel: 0.26


# Q.4

In [61]:
import tensorflow as tf

In [62]:
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

In [63]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [64]:
input_index = input_details[0]['index']
output_index = output_details[0]['index']

In [65]:
interpreter.set_tensor(input_index, img_array)

ValueError: Cannot set tensor: Dimension mismatch. Got 150 but expected 200 for dimension 1 of input 0.

In [66]:
interpreter.invoke()

In [67]:
output = interpreter.get_tensor(output_index)[0][0]
print(f"Output of the model: {output:.3f}")

Output of the model: 0.453
